In [1]:
from kaldi.decoder import LatticeFasterDecoder, LatticeFasterOnlineDecoder, LatticeFasterDecoderOptions, DecodableMatrixScaled
from kaldi.fstext import SymbolTable, StdVectorFst, read_fst_kaldi, utils
from kaldi.util.table import SequentialMatrixReader
from kaldi.matrix import Matrix

In [2]:
from kaldi.fstext.utils import get_linear_symbol_sequence

In [3]:
import re
import time
import pickle
import numpy as np

In [4]:
from edit_distance import SequenceMatcher
import torch
from neural_decoder.dataset import SpeechDataset

import matplotlib.pyplot as plt

In [5]:
from neural_decoder.neural_decoder_trainer import getDatasetLoaders
from neural_decoder.neural_decoder_trainer import loadModel

In [6]:
#not needed since not using LLM model
import torch
import huggingface_hub
from transformers import AutoTokenizer, AutoModelForCausalLM 

In [7]:
import torch.nn.functional as F
import os
import numpy as np
import math

In [8]:
class DecodableTensorScaled:
    def __init__(self):
        self.reset()
    def reset(self):
        self.num_frames_ready = 0
        self.logp = []
    def AcceptLoglikes(self, logp):
        self.num_frames_ready += 1
        self.logp.append(logp.numpy())


In [9]:
class CtcWfstBeamSearch:
    def __init__(self, fst_, opts: LatticeFasterDecoderOptions, symbol_table: SymbolTable, acoustic_scale, nbest):
        self.decodable = DecodableTensorScaled()
        self.decoder = LatticeFasterDecoder(fst_, opts) #LatticeFasterOnlineDecoder(fst_, opts)
        self.symbol_table = symbol_table
        self.opts = opts
        self.nbest = nbest
        self.blank_scale = 1
        self.blank_skip_thresh = 0.95
        self.blank = 1
        self.acoustic_scale = acoustic_scale
        self.reset()
        

    def reset(self):
        self.num_frames = 0
        self.decoded_frames_mapping = []
        self.last_best = 0
        self.inputs = []
        self.outputs = []
        self.likelihood = []
        self.times = []
        self.decodable.reset()
        self.is_last_frame_blank = False
        self.decoder.init_decoding()


    def decode_matrix(self):
        if self.decodable.num_frames_ready == 0:
            raise ValueError("No frames to decode!")
        m = Matrix(np.array(self.decodable.logp))
        print("shape: ", m.numpy().shape)
        dms = DecodableMatrixScaled(m, self.acoustic_scale)
        print("num indices: ", dms.num_indices())
        self.decoder.advance_decoding(dms)
            
    def search(self, logp):
        if(len(logp) == 0):
            return
        for i, frame_logp in enumerate(logp):
            blank_score = np.exp(frame_logp[self.blank])
            if blank_score > self.blank_skip_thresh * self.blank_scale:
                self.is_last_frame_blank = True
                self.last_frame_prob = frame_logp
            else:
                cur_best = np.argmax(frame_logp)
                if (cur_best != self.blank and self.is_last_frame_blank and cur_best == self.last_best):
                    self.decodable.AcceptLoglikes(self.last_frame_prob)
                    self.decoded_frames_mapping.append(self.num_frames - 1)
                self.last_best = cur_best
                self.decodable.AcceptLoglikes(frame_logp)
                self.decoded_frames_mapping.append(self.num_frames)
                self.is_last_frame_blank = False
            self.num_frames += 1   

        self.decode_matrix()
        self.inputs.clear()
        self.outputs.clear()
        self.likelihood.clear()
        print("decoded frames length: ", len(self.decoded_frames_mapping))
        if len(self.decoded_frames_mapping) > 0:
            if self.nbest == 1:
                self.inputs.append([])
                self.outputs.append([])
                self.likelihood.append(0)
                lat = self.decoder.get_best_path()
                alignment, words, weight = utils.get_linear_symbol_sequence(lat)
                print("words: ", words)
                self.convert_to_inputs(alignment)
                self.outputs[0] = words
                self.likelihood[0] = -(weight.value1 + weight.value2)
            else:
                lat2 = self.decoder.get_lattice()
                lat2_lat = utils.convert_compact_lattice_to_lattice(lat2)
                lat2_std = utils.convert_lattice_to_std(lat2_lat)
                nbest_fsts = utils.nbest_as_fsts(lat2_std, self.nbest)
                n = len(nbest_fsts)
                self.inputs = [[] for _ in range(n)]
                self.outputs = [[] for _ in range(n)]
                self.likelihood = [0.0 for _ in range(n)]
                self.times = [[] for _ in range(n)]
                for i, nbest_ in enumerate(nbest_fsts):
                    nbest_lat = utils.convert_std_to_lattice(nbest_)
                    alignment, words, weight = utils.get_linear_symbol_sequence(nbest_lat)
                    print("words: ", words)
                    self.convert_to_inputs(alignment, i)
                    self.outputs[i] = words
                    self.likelihood[i] = -(weight.value1 + weight.value2)


    def convert_to_inputs(self, alignment, i=0, time = None):
        self.inputs[i].clear()
        if time is not None:
            time.clear()
        for cur in range(len(alignment)):
            if alignment[cur]-1 == self.blank:
                continue
            if cur > 0 and alignment[cur] == alignment[cur-1]:
                continue
            self.inputs[i].append(alignment[cur]-1)
            if time is not None:
                time.append(self.decoded_frames_mapping[cur])


In [10]:
class DecodeResult:
    def __init__(self):
        self.lm_score = 0.0
        self.ac_score = 0.0
        self.sentence = ""

In [11]:
class PyKaldiDecoder:
    def __init__(self, model_path, acoustic_scale=0.5, nbest=10, beam=18):
        fst_path = os.path.join(model_path, "TLG.fst")
        if not os.path.exists(fst_path):
            raise ValueError(f"TLG.fst not found in {model_path}")
        self.fst = StdVectorFst.read(fst_path)
        self.acoustic_scale = acoustic_scale
        opts = LatticeFasterDecoderOptions()
        opts.beam = beam
        opts.max_active = 7000
        opts.min_active = 200
        opts.lattice_beam = 8
        self.symbol_table = SymbolTable.read_text(os.path.join(model_path, "words.txt"))
        self.decoder = CtcWfstBeamSearch(self.fst, opts, self.symbol_table, acoustic_scale, nbest)
        self.results = []
    def decode(self, logp):
        self.decoder.reset()
        self.decoder.search(logp)
        self.updateResult()
        return
    
    def updateResult(self):
        hypothesis = self.decoder.outputs
        likelihood = self.decoder.likelihood
        self.results.clear()

        assert len(hypothesis) == len(likelihood)
        for i, hypothesis in enumerate(hypothesis):
            dr = DecodeResult()
            dr.lm_score = likelihood[i] # might need to double check this?
            dr.ac_score = likelihood[i] / self.acoustic_scale

            for token in hypothesis:
                dr.sentence += f' {self.symbol_table.find_symbol(token)}'#_symbol(token)

            dr.sentence = dr.sentence.strip()
            self.results.append(dr)

        if (len(self.results) > 0 and bool(self.results[0].sentence)):
            print(f"Partial CTC result: {self.results[0].sentence}")
        return

    def get_results(self):
        return self.results


In [12]:
def rearrange_speech_logits(logits, has_sil=False):
    if not has_sil:
        logits = np.concatenate([logits[:, :, -1:], logits[:, :, :-1]], axis=-1)
    else:
        logits = np.concatenate([logits[:, :, -1:], logits[:, :, -2:-1], logits[:, :, :-2]], axis=-1)
    return logits

In [13]:
def lm_decode(pydecoder, logits, returnNBest=False, rescore=False, blankPenalty=0.0, logPriors=None):
    assert len(logits.shape) == 2
    logPriors = torch.from_numpy(np.zeros([1, logits.shape[1]]))
    log_probs = F.log_softmax(torch.from_numpy(logits), dim=-1)
    log_probs = log_probs - logPriors
    # apply blank penalty
    blank_log_probs = log_probs[:, 0:1]
    log_probs[:, 0:1] = blank_log_probs - blankPenalty
    pydecoder.decode(log_probs)
    results = pydecoder.results
    if returnNBest:
        decoded = []
        for r in results:
            decoded.append((r.sentence, r.ac_score, r.lm_score))
        return decoded
    else:
        return results[0].sentence #?


In [14]:
def wer_(r, h):
    """
    Calculation of WER with Levenshtein distance.
    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.
    Parameters
    ----------
    r : list
    h : list
    Returns
    -------
    int
    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
    d = d.reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion    = d[i][j-1] + 1
                deletion     = d[i-1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]

In [28]:
def build_opt(modelName='facebook/opt-6.7b', cacheDir=None, device='cpu', load_in_8bit=False):
    from transformers import AutoModelForCausalLM, AutoTokenizer

    tokenizer = AutoTokenizer.from_pretrained(modelName, cache_dir=cacheDir)
    model = AutoModelForCausalLM.from_pretrained(modelName, cache_dir=cacheDir,
                                                 device_map=device, load_in_8bit=load_in_8bit)

    tokenizer.padding_side = "right"
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [16]:
def rescore_with_gpt2(model, tokenizer, hypotheses, lengthPenalty):
    model_class = type(model).__name__
    if model_class.startswith('TF'):
        inputs = tokenizer(hypotheses, return_tensors='tf', padding=True)
        outputs = model(inputs)
        logProbs = tf.math.log(tf.nn.softmax(outputs['logits'], -1))
        logProbs = logProbs.numpy()
    else:
        import torch
        inputs = tokenizer(hypotheses, return_tensors='pt', padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
            logProbs = torch.nn.functional.log_softmax(outputs['logits'].float(), -1).numpy()

    newLMScores = []
    B, T, _ = logProbs.shape
    for i in range(B):
        n_tokens = np.sum(inputs['attention_mask'][i].numpy())

        newLMScore = 0.
        for j in range(1, n_tokens):
            newLMScore += logProbs[i, j - 1, inputs['input_ids'][i, j].numpy()]

        newLMScores.append(newLMScore - n_tokens * lengthPenalty)

    return newLMScores

In [17]:
def gpt2_lm_decode(model, tokenizer, nbest, acousticScale, lengthPenlaty, alpha,
                   returnConfidence=False):
    hypotheses = []
    acousticScores = []
    oldLMScores = []
    for out in nbest:
        hyp = out[0].strip()
        if len(hyp) == 0:
            continue
        hyp = hyp.replace('>', '')
        hyp = hyp.replace('  ', ' ')
        hyp = hyp.replace(' ,', ',')
        hyp = hyp.replace(' .', '.')
        hyp = hyp.replace(' ?', '?')
        hypotheses.append(hyp)
        acousticScores.append(out[1])
        oldLMScores.append(out[2])

    if len(hypotheses) == 0:
        return "" if not returnConfidence else ("", 0.)

    acousticScores = np.array(acousticScores)
    newLMScores = np.array(rescore_with_gpt2(model, tokenizer, hypotheses, lengthPenlaty))
    oldLMScores = np.array(oldLMScores)

    totalScores = alpha * newLMScores + (1 - alpha) * oldLMScores + acousticScale * acousticScores
    maxIdx = np.argmax(totalScores)
    bestHyp = hypotheses[maxIdx]
    if not returnConfidence:
        return bestHyp
    else:
        totalScores = totalScores - np.max(totalScores)
        probs = np.exp(totalScores)
        return bestHyp, probs[maxIdx] / np.sum(probs)

In [33]:
def cer_with_gpt2_decoder(model, tokenizer, nbestOutputs, acousticScale,
                          inferenceOut, outputType='handwriting',
                          returnCI=False,
                          lengthPenalty=0.0,
                          alpha=1.0):
    decodedSentences = []
    confidences = []
    for i in range(len(nbestOutputs)):
        print("it: ", i)
        decoded, confidence = gpt2_lm_decode(model, tokenizer, nbestOutputs[i], acousticScale, lengthPenalty, alpha, returnConfidence=True)
        decodedSentences.append(decoded)
        print("decoded: ", decoded)
        confidences.append(confidence)

    if outputType == 'handwriting':
        trueSentences = _extract_true_sentences(inferenceOut)
    elif outputType == 'speech' or outputType == 'speech_sil':
        trueSentences = _extract_transcriptions(inferenceOut)
    trueSentencesProcessed = []
    for trueSent in trueSentences:
        if outputType == 'handwriting':
            trueSent = trueSent.replace('>',' ')
            trueSent = trueSent.replace('~','.')
            trueSent = trueSent.replace('#','')
        if outputType == 'speech' or outputType == 'speech_sil':
            trueSent = trueSent.strip()
        trueSentencesProcessed.append(trueSent)

    cer, wer = _cer_and_wer(decodedSentences, trueSentencesProcessed, outputType, returnCI)

    return {
        'cer': cer,
        'wer': wer,
        'decoded_transcripts': decodedSentences,
        'confidences': confidences
    }

In [19]:
def _cer_and_wer(decodedSentences, trueSentences, outputType='handwriting',
                 returnCI=False):
    allCharErr = []
    allChar = []
    allWordErr = []
    allWord = []
    for x in range(len(decodedSentences)):
        decSent = decodedSentences[x]
        trueSent = trueSentences[x]

        nCharErr = wer_([c for c in trueSent], [c for c in decSent])
        if outputType == 'handwriting':
            trueWords = trueSent.replace(">", " > ").split(" ")
            decWords = decSent.replace(">", " > ").split(" ")
        elif outputType == 'speech' or outputType == 'speech_sil':
            trueWords = trueSent.split(" ")
            decWords = decSent.split(" ")
        nWordErr = wer_(trueWords, decWords)

        allCharErr.append(nCharErr)
        allWordErr.append(nWordErr)
        allChar.append(len(trueSent))
        allWord.append(len(trueWords))

    cer = np.sum(allCharErr) / np.sum(allChar)
    wer = np.sum(allWordErr) / np.sum(allWord)

    if not returnCI:
        return cer, wer
    else:
        allChar = np.array(allChar)
        allCharErr = np.array(allCharErr)
        allWord = np.array(allWord)
        allWordErr = np.array(allWordErr)

        nResamples = 10000
        resampledCER = np.zeros([nResamples,])
        resampledWER = np.zeros([nResamples,])
        for n in range(nResamples):
            resampleIdx = np.random.randint(0, allChar.shape[0], [allChar.shape[0]])
            resampledCER[n] = np.sum(allCharErr[resampleIdx]) / np.sum(allChar[resampleIdx])
            resampledWER[n] = np.sum(allWordErr[resampleIdx]) / np.sum(allWord[resampleIdx])
        cerCI = np.percentile(resampledCER, [2.5, 97.5])
        werCI = np.percentile(resampledWER, [2.5, 97.5])

        return (cer, cerCI[0], cerCI[1]), (wer, werCI[0], werCI[1])

In [20]:
def _extract_transcriptions(inferenceOut):
#     transcriptions = []
#     for i in range(len(inferenceOut['transcriptions'])):
#         endIdx = np.argwhere(inferenceOut['transcriptions'][i] == 0)[0, 0]
#         trans = ''
#         for c in range(endIdx):
#             trans += chr(inferenceOut['transcriptions'][i][c])
#         transcriptions.append(trans)

    return inferenceOut['transcriptions']

In [21]:
def _extract_true_sentences(inferenceOut):
    charMarks = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
                 '>',',',"'",'~','?']

    trueSentences = []
    for i in range(len(inferenceOut['trueSeqs'])):
        trueSent = ''
        endIdx = np.argwhere(inferenceOut['trueSeqs'][i] == -1)
        endIdx = endIdx[0,0]
        for c in range(endIdx):
            trueSent += charMarks[inferenceOut['trueSeqs'][i][c]]
        trueSentences.append(trueSent)

    return trueSentences

In [22]:
def cer_pre_opt(nbestOutputs, inferenceOut):
    decoded_sentences = []
    true_sentences_processed = []
    true_sentences = inferenceOut['transcriptions'] #_extract_transcriptions(inferenceOut)
    for i in range(len(nbestOutputs)):
        sentence = nbestOutputs[i][0][0]
        hyp = sentence.strip()
        hyp = hyp.replace('>', '')
        hyp = hyp.replace('  ', ' ')
        hyp = hyp.replace(' ,', ',')
        hyp = hyp.replace(' .', '.')
        hyp = hyp.replace(' ?', '?')
        decoded_sentences.append(hyp)
        trueSent = true_sentences[i].strip()
        true_sentences_processed.append(trueSent)
    cer, wer = _cer_and_wer(decoded_sentences, true_sentences_processed, "speech_sil")
    return cer, wer
        

In [14]:
args = {}
# args['outputDir'] = '/oak/stanford/groups/henderj/stfan/logs/speech_logs/' + modelName
args['datasetPath'] = '/home/iris/project_3_bci/workload_characterization/id20_neural_decode/data/competition_data/ptDecoder_ctc'
args['seqLen'] = 150
args['maxTimeSeriesLen'] = 1200
args['batchSize'] = 8 #64
args['lrStart'] = 0.02
args['lrEnd'] = 0.02
args['nUnits'] = 1024
args['nBatch'] = 10000 #3000
args['nLayers'] = 5
args['seed'] = 0
args['nClasses'] = 40
args['nInputFeatures'] = 256
args['dropout'] = 0.4
args['whiteNoiseSD'] = 0.8
args['constantOffsetSD'] = 0.2
args['gaussianSmoothWidth'] = 2.0
args['strideLen'] = 4
args['kernelLen'] = 32
args['bidirectional'] = True
args['l2_decay'] = 1e-5

In [15]:
trainLoaders, testLoaders, loadedData = getDatasetLoaders(
    args["datasetPath"], args["batchSize"]
)

In [16]:
modelPath = '/home/iris/project_3_bci/workload_characterization/id20_neural_decode/model/speechBaseline4'
model = loadModel(modelPath, device="cpu")
device = "cpu"
model.eval()

/home/iris/.local/lib/python3.10/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


GRUDecoder(
  (inputLayerNonlinearity): Softsign()
  (unfolder): Unfold(kernel_size=(32, 1), dilation=1, padding=0, stride=4)
  (gaussianSmoother): GaussianSmoothing()
  (gru_decoder): GRU(8192, 1024, num_layers=5, batch_first=True, dropout=0.4, bidirectional=True)
  (inpLayer0): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer1): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer2): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer3): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer4): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer5): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer6): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer7): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer8): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer9): Linear(in_features=256, out_features=256, bias=True)
  (inpLayer10): Linear(in_features=256, out_features=2

In [17]:
rnn_outputs = {
    "logits": [],
    "logitLengths": [],
    "trueSeqs": [],
    "transcriptions": [],
}

# partition = "competition" 
partition =  "test"
if partition == "competition":
    testDayIdxs = [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20]
elif partition == "test":
    testDayIdxs = range(len(loadedData[partition]))

for i, testDayIdx in enumerate(testDayIdxs):
    test_ds = SpeechDataset([loadedData[partition][i]])
    test_loader = torch.utils.data.DataLoader(
        test_ds, batch_size=1, shuffle=False, num_workers=0
    )
    for j, (X, y, X_len, y_len, _) in enumerate(test_loader):
        X, y, X_len, y_len, dayIdx = (
            X.to(device),
            y.to(device),
            X_len.to(device),
            y_len.to(device),
            torch.tensor([testDayIdx], dtype=torch.int64).to(device),
        )
        pred = model.forward(X, dayIdx)
        adjustedLens = ((X_len - model.kernelLen) / model.strideLen).to(torch.int32)

        for iterIdx in range(pred.shape[0]):
            trueSeq = np.array(y[iterIdx][0 : y_len[iterIdx]].cpu().detach())

            rnn_outputs["logits"].append(pred[iterIdx].cpu().detach().numpy())
            rnn_outputs["logitLengths"].append(
                adjustedLens[iterIdx].cpu().detach().item()
            )
            rnn_outputs["trueSeqs"].append(trueSeq)

        transcript = loadedData[partition][i]["transcriptions"][j].strip()
        transcript = re.sub(r"[^a-zA-Z\- \']", "", transcript)
        transcript = transcript.replace("--", "").lower()
        rnn_outputs["transcriptions"].append(transcript)


/home/iris/project_3_bci/workload_characterization/id20_neural_decode/neural_seq_decoder/src/neural_decoder/augmentations.py:91: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return self.conv(input, weight=self.weight, groups=self.groups, padding="same")


In [18]:
# print(dir(LatticeFasterDecoderOptions))

In [19]:
# print(dir(DecodableTensorScaled))

In [20]:
lmDir = "/home/iris/project_3_bci/workload_characterization/id20_neural_decode/model/languageModel"
ngramDecoder = PyKaldiDecoder(lmDir)

In [21]:
# LM decoding hyperparameters
acoustic_scale = 0.5
blank_penalty = np.log(7)
llm_weight = 0.5

llm_outputs = []
# Generate nbest outputs from 5gram LM
start_t = time.time()
nbest_outputs = []
for j in range(len(rnn_outputs["logits"])):
# for j in range(1):
    logits = rnn_outputs["logits"][j]
    logits = np.concatenate(
        [logits[:, 1:], logits[:, 0:1]], axis=-1
    )  # Blank is last token
    logits = rearrange_speech_logits(logits[None, :, :], has_sil=True)
    nbest = lm_decode(
        ngramDecoder,
        logits[0],
        blankPenalty=blank_penalty,
        returnNBest=True,
        rescore=True,
    )
    nbest_outputs.append(nbest)
# time_per_sample = (time.time() - start_t) / len(rnn_outputs["logits"])
# print(f"decoding took {time_per_sample} seconds per sample")

shape:  (35, 41)
num indices:  41
decoded frames length:  35
words:  [112089, 99727]
words:  [101577, 91996]
words:  [112089, 468]
words:  [101577, 93799]
words:  [49337, 93799]
words:  [49337, 91996]
words:  [112257, 4937]
words:  [112367, 122428]
words:  [112089, 37109, 23909]
words:  [112257, 108851]
Partial CTC result: THE SCREEN
shape:  (65, 41)
num indices:  41
decoded frames length:  65


LOG ([5.5.952~1-44757]:RebuildRepository():determinize-lattice-pruned.cc:287) Rebuilding repository.


words:  [93813, 48238, 112089, 37251]
words:  [93813, 48238, 112089, 17617]
words:  [93813, 16563, 65075, 112089, 95790]
words:  [93813, 48238, 112089, 96177]
words:  [93813, 48238, 112089, 115668]
words:  [36945, 49168, 97277, 53748, 112089, 115668]
words:  [93813, 3653, 65075, 112089, 95790]
words:  [36945, 49168, 97277, 53748, 112089, 37251]
words:  [36945, 49168, 97277, 53748, 112089, 117238]
words:  [36945, 49168, 97277, 112063, 112089, 115668]
Partial CTC result: RESPONSIVE HANDLING THE F
shape:  (76, 41)
num indices:  41
decoded frames length:  76
words:  [112089, 96628, 4912, 67691, 54319, 3, 104867]
words:  [112089, 96628, 4912, 67691, 54319, 115875, 21549]
words:  [112257, 122074, 8630, 67691, 54319, 3, 104867]
words:  [112089, 96628, 4912, 67691, 54319, 55641, 104867]
words:  [112089, 96628, 4912, 67691, 54319, 54319, 3, 104867]
words:  [112089, 96628, 121084, 67691, 54319, 3, 104867]
words:  [112257, 122074, 8630, 67691, 54319, 115875, 21549]
words:  [112089, 96628, 120518,

LOG ([5.5.952~1-44757]:RebuildRepository():determinize-lattice-pruned.cc:287) Rebuilding repository.


words:  [91482, 20069, 113151, 68142, 51016, 303]
words:  [91482, 20069, 113151, 68142, 51016, 93382, 2519, 26792]
words:  [91482, 20069, 113151, 68142, 51016, 256, 2519, 26792]
words:  [91482, 40092, 1535, 113151, 68142, 51016, 303]
words:  [91482, 20069, 21776, 51016, 303]
words:  [91482, 20069, 113151, 68142, 51016, 3, 121668, 26792]
words:  [91482, 20069, 21776, 103066, 2519, 26792]
words:  [91482, 20069, 113151, 68142, 51016, 117092, 2519, 26792]
words:  [91482, 40092, 1535, 113151, 68142, 51016, 93382, 2519, 26792]
words:  [91482, 40092, 1535, 113151, 68142, 51016, 256, 2519, 26792]
Partial CTC result: READABLE CINNAMON TO MAKE HIM ABRUPTLY
shape:  (85, 41)
num indices:  41
decoded frames length:  85
words:  [110615, 256, 3524, 6314, 107931, 54319, 115668]
words:  [110615, 256, 3524, 6314, 107780, 115668]
words:  [110615, 256, 3524, 6314, 113151, 34934, 115668]
words:  [53551, 110615, 256, 3524, 6314, 107931, 54319, 115668]
words:  [110615, 256, 3524, 6314, 107931, 3653, 115668]


...

words:  [38883, 59769, 84409, 42411]
words:  [38883, 21902, 18599]
words:  [38883, 21902, 16563]
words:  [38883, 21811, 124047, 16563]
words:  [38883, 21902, 45740]
words:  [38883, 21803, 124047, 16563]
words:  [38883, 21902, 20300]
words:  [38883, 21902, 124047, 16563]
words:  [38883, 59769, 124047, 16563]
words:  [110276, 21803, 124047, 16563]
Partial CTC result: FIFTEEN KILOMETERS PER GALLON
shape:  (22, 41)
num indices:  41
decoded frames length:  22
words:  [54319, 56462]
words:  [112149, 54319, 56462]
words:  [112038, 54319, 56462]
words:  [80542, 54319, 56462]
words:  [54319, 54319, 56462]
words:  [56462]
words:  [112331, 56450, 2519]
words:  [54319, 28961]
words:  [112149, 56450, 2519]
words:  [3653, 54319, 56462]
Partial CTC result: IN ITALY
shape:  (35, 41)
num indices:  41
decoded frames length:  35
words:  [56450, 102299, 8630, 27169]
words:  [56450, 24033, 8630, 27169]
words:  [56450, 123194, 8630, 27169]
words:  [56450, 102299, 8630, 28942]
words:  [56450, 102299, 8630, 2

LOG ([5.5.952~1-44757]:RebuildRepository():determinize-lattice-pruned.cc:287) Rebuilding repository.


words:  [53748, 53551, 31415, 49168, 3, 91531, 36516, 80517, 56450]
words:  [53748, 53551, 31415, 49168, 3, 91531, 34160, 80517, 56450]
words:  [53748, 53551, 31415, 49168, 3, 91531, 122852, 80517, 56450]
words:  [53748, 53551, 31415, 49168, 3, 91531, 34160, 113438]
words:  [53748, 53551, 31415, 49168, 3, 121386, 122580, 15806, 80517, 56450]
words:  [53748, 53551, 31415, 49168, 3, 91531, 122580, 3, 15801, 80517, 56450]
words:  [53748, 53551, 31415, 49168, 3, 91531, 34160, 54319, 56450]
words:  [53748, 53551, 31415, 49168, 3, 91531, 122580, 4487, 80517, 56450]
words:  [53748, 53551, 31415, 49168, 3, 91531, 120542, 80517, 56450]
words:  [53748, 53551, 31415, 49168, 3, 91531, 34163, 54319, 56450]
Partial CTC result: IF I DON'T HAVE A REAL EVIDENCE ON IT
shape:  (42, 41)
num indices:  41
decoded frames length:  42
words:  [124047, 43780, 3, 66504, 79959, 41890, 37355]
words:  [124047, 43780, 3, 66504, 79959, 23136]
words:  [124047, 43780, 3, 66504, 79959, 41890, 90440]
words:  [124047, 437

decoded frames length:  60
words:  [53551, 120319, 36431, 109451, 113151, 85556, 117016]
words:  [53551, 120319, 36431, 109147, 113151, 105504, 79959]
words:  [53551, 120319, 36431, 109147, 113151, 105504, 117016]
words:  [53551, 120319, 36431, 109451, 113151, 105504, 79959]
words:  [53551, 120319, 36431, 109451, 113151, 105504, 117016]
words:  [53551, 65753, 54319, 3, 10394, 113151, 85556, 117016]
words:  [53551, 65753, 54319, 3, 10394, 113151, 8885]
words:  [53551, 65753, 54319, 112721, 113151, 85604]
words:  [53551, 122074, 63810, 4330, 21845, 113151, 85604]
words:  [53551, 120319, 36431, 22014, 113151, 105504, 79959]
Partial CTC result: I WASN'T EVEN SUPPOSED TO PICK UP
shape:  (39, 41)
num indices:  41
decoded frames length:  39
words:  [56450, 91569, 112063, 112493, 84364]
words:  [56450, 91569, 64582, 112493, 84364]
words:  [56450, 122074, 64582, 112493, 84364]
words:  [56450, 91569, 64604, 112493, 84364]
words:  [56450, 91569, 51252, 112493, 84364]
words:  [56450, 91569, 49043,

decoded frames length:  33
words:  [53551, 77581, 36431, 60533, 112063]
words:  [53551, 66622, 36431, 30913, 112063]
words:  [53551, 77581, 36431, 30913, 112063]
words:  [53853, 36431, 30913, 112063]
words:  [53551, 36431, 30913, 112063]
words:  [53551, 65753, 3653, 30913, 112063]
words:  [77581, 36431, 60533, 112063]
words:  [53551, 77581, 36452, 30913, 112063]
words:  [53551, 66622, 56450, 30913, 112063]
words:  [53551, 66622, 3653, 30913, 112063]
Partial CTC result: I NEVER EVEN KNEW THAT
shape:  (33, 41)
num indices:  41
decoded frames length:  33
words:  [20707, 17617]
words:  [26530, 97004]
words:  [20707, 123868]
words:  [20707, 97004]
words:  [20707, 17639]
words:  [21407, 3653, 19738]
words:  [20707, 97277]
words:  [20707, 46962]
words:  [65367, 32052]
words:  [20707, 60117]
Partial CTC result: CLINTON CASE
shape:  (33, 41)
num indices:  41
decoded frames length:  33
words:  [112089, 103053, 110977]
words:  [112089, 21849, 30730]
words:  [112089, 21849, 101146]
words:  [112089

In [22]:
nbest_outputs

[[('THE SCREEN', -110.2320556640625, -55.11602783203125),
  ('SHE RECOUNTED', -110.65758514404297, -55.328792572021484),
  ('THE ACCIDENT', -110.9535903930664, -55.4767951965332),
  ('SHE RESPONDED', -112.22803497314453, -56.114017486572266),
  ('HE RESPONDED', -112.2502212524414, -56.1251106262207),
  ('HE RECOUNTED', -112.27938079833984, -56.13969039916992),
  ("THEY AREN'T", -112.41215515136719, -56.206077575683594),
  ('THIS WINTER', -113.7474365234375, -56.87371826171875),
  ('THE EXTRA COST', -113.76994323730469, -56.884971618652344),
  ('THEY SUCCEEDED', -113.9569091796875, -56.97845458984375)],
 [('RESPONSIVE HANDLING THE F', -177.6196746826172, -88.8098373413086),
  ('RESPONSIVE HANDLING THE CASE', -178.17660522460938, -89.08830261230469),
  ('RESPONSIVE CAN LIFT THE ROOF', -178.4032440185547, -89.20162200927734),
  ('RESPONSIVE HANDLING THE ROUGH', -178.46482849121094, -89.23241424560547),
  ('RESPONSIVE HANDLING THE U', -178.5130157470703, -89.25650787353516),
  ('EXPERTS HA

In [53]:
rnn_outputs["transcriptions"]

['theocracy reconsidered',
 'rich purchased several signed lithographs',
 'so rules we made in unabashed collusion',
 "lori's costume needed black gloves to be completely elegant",
 "the tooth fairy forgot to come when roger's tooth fell out",
 'that stinging vapor was caused by chloride vaporization',
 "before thursday's exam review every formula",
 'wildfire near sunshine forces park closures',
 "the word means it won't boil away easily nothing else",
 "would a blue feather in a man's hat make him happy all day",
 'he talked about unauthentic storylines too',
 'most young rise early every morning',
 'with this no loyal citizen can quarrel',
 'primitive tribes have an upbeat attitude',
 'and it was not a judge but a justice of the peace who made the decision',
 'we seemed to be witnessing the population explosion right in our own backyards',
 'sometimes desperate measures can also turn into solid moves for the future',
 'those answers will be straightforward if you think them through 

In [103]:
cer_pre_opt(nbest_outputs, rnn_outputs)

(0.8455420374069367, 1.0029096199308964)

In [71]:
# #write to pkl object if doing llm separately
# with open("rnn_outputs.pkl", "wb") as f:
#     pickle.dump(rnn_outputs, f)
# with open("nbest_outputs.pkl", "wb") as f:
#     pickle.dump(nbest_outputs, f)

In [72]:
# # read rnn_ouputs and nbest_outputs if doing llm separately
# with open("rnn_outputs.pkl", "rb") as f:
#     rnn_outputs = pickle.load(f)
# with open("nbest_outputs.pkl", "rb") as f:
#     nbest_outputs = pickle.load(f)

In [29]:
llm, llm_tokenizer = build_opt()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
llm_out = cer_with_gpt2_decoder(
    llm,
    llm_tokenizer,
    nbest_outputs[:],
    0.5,
    rnn_outputs,
    outputType="speech_sil",
    returnCI=False,
    lengthPenalty=0,
    alpha=0.5,
)

it:  0
decoded:  THE SCREEN
it:  1
decoded:  EXPERTS HAVE SAID THAT THE U
it:  2
decoded:  THE RULES ARE MADE IN A SOLUTION
it:  3
decoded:  THE TEAM DID MAKE LOVE TO BE COMPLETELY ALONE
it:  4
decoded:  THE TRUTH VERY DIFFICULT TO MANAGE TO FILL OUT
it:  5
decoded:  THAT THING WHERE I WAS TOLD THE COLLEGE PREPARATION
it:  6
decoded:  BY FORCEFULLY CONDEMNING WHEREVER POSSIBLE
it:  7
decoded:  WILD NEW INFRASTRUCTURE
it:  8
decoded:  THE WORD MEANS IT WENT REALLY WELL NOTHING ELSE
it:  9
decoded:  READABLE CINNAMON TO MAKE HIM ABRUPTLY
it:  10
decoded:  TALK ABOUT AN AUTHENTIC STORY IN U
it:  11
decoded:  STARTING WITH EARLY EVERY MORNING
it:  12
decoded:  WITH SO LITTLE CHILDREN CAN KILL
it:  13
decoded:  IF YOU HAVE AN UPBEAT ATTITUDE
it:  14
decoded:  AND WITH NO AGENDA BUT A CHOICE OF THE PRIEST WHO MADE THE DECISION
it:  15
decoded:  WE SEEM TO BE RUNNING THE RUSSIA COLLUSION IMPLICIT IN OUR OWN BACKYARD
it:  16
decoded:  COMBINES DISPARATE MEASURES AND ALSO IN A MOVE FOR THE FUTU

decoded:  SHE THOUGHT FOR A FEW MINUTES
it:  162
decoded:  THAT IS A SAD SITUATION
it:  163
decoded:  THEY DO HAVE CAPITAL PUNISHMENT
it:  164
decoded:  IT WAS NICE TALKING TO YOU TODAY
it:  165
decoded:  MY FAVORITE GAME IS THE RANGE
it:  166
decoded:  KIND OF GO ALONG WITH WHAT WE'RE SAYING
it:  167
decoded:  THEY WENT WITH PART ONE
it:  168
decoded:  CHOICE AWARD
it:  169
decoded:  THEY ARE THE DIFFERENT ELEMENTS
it:  170
decoded:  I WANT TO BRING A WOMAN AND SOME BACON
it:  171
decoded:  AS ANY CITY OR A
it:  172
decoded:  SO YOU'D BUY A PLANE
it:  173
decoded:  IT'S ABOUT SIX AND A HALF PERCENT
it:  174
decoded:  I AGREE WITH THE PHONE
it:  175
decoded:  SO I THINK IT'S GOING TO BE A LOT EASIER
it:  176
decoded:  NOT SO MUCH TO US
it:  177
decoded:  WE TAKE A CERTAIN PORTION OF YOUR BACK
it:  178
decoded:  FUCK YOU
it:  179
decoded:  AND I DON'T KNOW WHAT'S GOING TO HAPPEN
it:  180
decoded:  I THINK WE HAVE ABOUT THREE MONTHS LEFT ON IT
it:  181
decoded:  WHY DO PEOPLE FIND IT IN


decoded:  STILL NO
it:  333
decoded:  TWO MAJOR PROBLEMS FOR THE LONG TERM
it:  334
decoded:  ARE THERE ANY OTHER COUNTRY SOUND ENGINEER EXPLAINED
it:  335
decoded:  I CAN'T REMEMBER WHAT THEY CALL A FULL TIME
it:  336
decoded:  FOR FAMILY REASONS
it:  337
decoded:  DO YOU KNOW THAT I'VE NEVER SEEN A MONKEY
it:  338
decoded:  I KNOW MOST PEOPLE
it:  339
decoded:  I RECOMMEND THE HUNTER TO A LOT OF PEOPLE
it:  340
decoded:  DO YOU HAVE THE MONEY FOR YOURSELF AT HOME
it:  341
decoded:  IT'S THE NO
it:  342
decoded:  YOU CAN BLAME THEM
it:  343
decoded:  MORE MARKETABLE
it:  344
decoded:  GUEST TRAFFIC COULD BE UNDONE
it:  345
decoded:  WE'VE GONE FOR MANY FOOD AND LIVELIHOOD
it:  346
decoded:  I HAD TO STAND IN
it:  347
decoded:  FOOTBALL IS AN EXCITING PART
it:  348
decoded:  I GOT IT ON
it:  349
decoded:  I DON'T KNOW THE DETAILS
it:  350
decoded:  WHEN YOU TALK MORE ABOUT THE SEVERE ASSIMILATION
it:  351
decoded:  SO MANY HORROR STORIES OUT THERE
it:  352
decoded:  EVEN IT HAS A THEOR

decoded:  I HAVEN'T EXPERIENCED IT
it:  515
decoded:  YOU HAVE TO HAVE A PRETTY GOOD TEAM
it:  516
decoded:  THE STORY ARCS AND LIKE THAT
it:  517
decoded:  THEY DON'T HAVE THE ABILITY TO SELL IT
it:  518
decoded:  WE GOT TO HAVE ONE DOESN'T
it:  519
decoded:  THERE'S SO MUCH GOING ON HERE
it:  520
decoded:  THE VICTIM'S FAMILY AND THINGS
it:  521
decoded:  NONE OF THIS LATE IN LIFE MOTHER
it:  522
decoded:  THE DRUG DEALER
it:  523
decoded:  MY WIFE'S MOTHER IS THE ONLY INNOCENT LIFE
it:  524
decoded:  THERE NOW
it:  525
decoded:  I HAVE NEVER SEEN BEFORE
it:  526
decoded:  I DON'T KNOW WHAT WE'RE GOING TO DO
it:  527
decoded:  IT'S SECOND NATURE
it:  528
decoded:  WHEN IT WAS THE LONGEST
it:  529
decoded:  WE DON'T GET TO SEE TOO MANY
it:  530
decoded:  IT'S GOT A LOT GOING FOR
it:  531
decoded:  I MEAN WHAT IT WAS IN DOLLARS
it:  532
decoded:  IT'S REALLY IMPORTANT
it:  533
decoded:  THE NEXT MORNING YOU WANT TO KICK THEM OUT
it:  534
decoded:  IF THE NEW WORK IS ANY GOOD
it:  535
d

decoded:  GOOD DUALISM
it:  697
decoded:  BUT IT WOULD PROBABLY WORK OUT
it:  698
decoded:  IT'S LITERALLY GOING TO EVOLVE NEW TRAITS
it:  699
decoded:  IT'S JUST GOING TO FALL
it:  700
decoded:  I'M KIND OF A CRIMINAL
it:  701
decoded:  BUYING A PIECE
it:  702
decoded:  SOMETIMES IT WILL STOP
it:  703
decoded:  I JUST DON'T LIKE THAT
it:  704
decoded:  USUALLY WILLING TO TALK ABOUT
it:  705
decoded:  I'M LOST IN MY UNCLE JERRY
it:  706
decoded:  EVERYTHING YOU NEED
it:  707
decoded:  MENTALLY KICKED HIMSELF
it:  708
decoded:  THEY'RE ACTUALLY WAITING FOR THEM
it:  709
decoded:  IT WAS FOR A MURDER TRIAL
it:  710
decoded:  ARE YOU A PROFESSIONAL MUSICIAN
it:  711
decoded:  IT'S FAR BEYOND ME
it:  712
decoded:  WE'VE MADE AN AWFUL LOT FROM THE SPACE PROGRAM
it:  713
decoded:  IT'S A COINCIDENCE
it:  714
decoded:  I CALL AND FIND OUT
it:  715
decoded:  YOU WON'T TELL YOU WHAT'S RIGHT
it:  716
decoded:  I'VE HEARD ABOUT HIM
it:  717
decoded:  WE'VE HAD MORE RAIN THIS WINTER
it:  718
decod

decoded:  CONTACT US FOR THE STATE
it:  877
decoded:  IF YOU HAVE ANY TYPE WHATSOEVER
it:  878
decoded:  JEREMY
it:  879
decoded:  I CAN'T REMEMBER


In [36]:
llm_out['decoded_transcripts']

['THE SCREEN',
 'EXPERTS HAVE SAID THAT THE U',
 'THE RULES ARE MADE IN A SOLUTION',
 'THE TEAM DID MAKE LOVE TO BE COMPLETELY ALONE',
 'THE TRUTH VERY DIFFICULT TO MANAGE TO FILL OUT',
 'THAT THING WHERE I WAS TOLD THE COLLEGE PREPARATION',
 'BY FORCEFULLY CONDEMNING WHEREVER POSSIBLE',
 'WILD NEW INFRASTRUCTURE',
 'THE WORD MEANS IT WENT REALLY WELL NOTHING ELSE',
 'READABLE CINNAMON TO MAKE HIM ABRUPTLY',
 'TALK ABOUT AN AUTHENTIC STORY IN U',
 'STARTING WITH EARLY EVERY MORNING',
 'WITH SO LITTLE CHILDREN CAN KILL',
 'IF YOU HAVE AN UPBEAT ATTITUDE',
 'AND WITH NO AGENDA BUT A CHOICE OF THE PRIEST WHO MADE THE DECISION',
 'WE SEEM TO BE RUNNING THE RUSSIA COLLUSION IMPLICIT IN OUR OWN BACKYARD',
 'COMBINES DISPARATE MEASURES AND ALSO IN A MOVE FOR THE FUTURE',
 'IT WILL BE DECIDED IS IF YOU THINK OF THE POLICE FORCE',
 'HE GETS REFRESHED AND RELAXED AND REKINDLING DECISION',
 'COMING SOON IN YOUR DIPLOMA',
 'TO SOME EXTENT PROCESSES ARE SHAPED BY EXPOSURE TO GROUP INVOLVEMENT',
 'A

In [43]:
for i in range(len(nbest_outputs)):
    real = rnn_outputs['transcriptions'][i]
    lm = nbest_outputs[i][0][0]
    llm = llm_out['decoded_transcripts'][i]
    print("Real:\n\t", real)
    print("Outputs:")
    print("\t LM: ", lm)
    print("\t LLM: ", llm)

Real:
	 theocracy reconsidered
Outputs:
	 LM:  THE SCREEN
	 LLM:  THE SCREEN
Real:
	 rich purchased several signed lithographs
Outputs:
	 LM:  RESPONSIVE HANDLING THE F
	 LLM:  EXPERTS HAVE SAID THAT THE U
Real:
	 so rules we made in unabashed collusion
Outputs:
	 LM:  THE RULES ARE MADE IN A SOLUTION
	 LLM:  THE RULES ARE MADE IN A SOLUTION
Real:
	 lori's costume needed black gloves to be completely elegant
Outputs:
	 LM:  THE TEAM DID MAKE LOVE TO BE COMPLETELY ALONE
	 LLM:  THE TEAM DID MAKE LOVE TO BE COMPLETELY ALONE
Real:
	 the tooth fairy forgot to come when roger's tooth fell out
Outputs:
	 LM:  THE TRUTH VERY RUDE TO SOMEONE WERE TO FALL OUT
	 LLM:  THE TRUTH VERY DIFFICULT TO MANAGE TO FILL OUT
Real:
	 that stinging vapor was caused by chloride vaporization
Outputs:
	 LM:  THAT IN FACT I WAS TOLD THE SITE PREPARATION
	 LLM:  THAT THING WHERE I WAS TOLD THE COLLEGE PREPARATION
Real:
	 before thursday's exam review every formula
Outputs:
	 LM:  BY FORCEFULLY CONDEMNING WHEREVER